# Sales Forecasting Project
**Project by Veer Wakhariya**

# Retail Sales Forecasting (Weekly)

This notebook performs EDA and a 12-week forecast using Holt-Winters (Exponential Smoothing).

In [ ]:
import pandas as pd, matplotlib.pyplot as plt
weekly_total = pd.read_csv('weekly_aggregations.csv', parse_dates=['date'])
weekly_total.head()

In [ ]:
# Plot weekly sales and rolling mean
plt.figure(figsize=(10,5))
plt.plot(weekly_total['date'], weekly_total['sales'], label='Weekly Sales')
plt.plot(weekly_total['date'], weekly_total['rolling_8w'], label='8-week Rolling Mean')
plt.title('Weekly Sales & Trend'); plt.xlabel('Week'); plt.ylabel('Sales'); plt.legend(); plt.tight_layout(); plt.show()

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
ts = weekly_total.set_index('date')['sales'].asfreq('W-SUN')
model = ExponentialSmoothing(ts, trend='add', seasonal='add', seasonal_periods=52, initialization_method='estimated')
fit = model.fit()
pred = fit.forecast(12)
import numpy as np
conf_int = 1.96*np.std(fit.resid, ddof=1)
f_index = pd.date_range(ts.index[-1]+pd.Timedelta(weeks=1), periods=12, freq='W-SUN')
forecast_df = pd.DataFrame({'date': f_index, 'forecast': pred.values,
                            'lower_ci': pred.values-conf_int,
                            'upper_ci': pred.values+conf_int})
forecast_df.head()

In [ ]:
# Plot actual vs forecast
plt.figure(figsize=(10,5))
start = ts.index[-60] if len(ts)>60 else ts.index[0]
plt.plot(ts.loc[start:], label='Actual')
plt.plot(f_index, pred.values, label='Forecast')
plt.fill_between(f_index, pred.values-conf_int, pred.values+conf_int, alpha=0.2, label='95% CI')
plt.title('Actual vs Forecast (Next 12 Weeks)'); plt.xlabel('Week'); plt.ylabel('Sales'); plt.legend(); plt.tight_layout(); plt.show()

In [ ]:
weekly_total.to_csv('weekly_aggregations.csv', index=False)
forecast_df.to_csv('weekly_forecast_12w.csv', index=False)
print('Saved weekly_aggregations.csv and weekly_forecast_12w.csv')